In [1]:
import numpy as np
from scipy.io import loadmat
file_name = '/mnt/data/WQ/Raw-CSI-Data/ArgosCSI-96x8-2016-11-04-05-37-37_2.4GHz_track_left_to_right_NLOS.npy'
file_name2 = '/mnt/data/WQ/Raw-CSI-Data/ArgosCSI-96x8-2016-05-01-06-57-58-2.4GHz-continuousmobile.npy'

file_name3 = '/mnt/data/WQ/Raw-CSI-Data/ArgosCSI-96x2-2016-12-07-03-00-36_rotation_mob_horizontal_omni.npy'

file_name4 = '/mnt/data/WQ/Argos/train_18_test_6_antenna_96_subcarriers_52/train/downlink/ArgosCSI-96x8-2016-05-01-07-17-44-5GHz-continuousmobility.mat'
file_name5 = '/mnt/data/WQ/Argos/train_18_test_6_antenna_96_subcarriers_52/train/downlink/ArgosCSI-96x2-2016-12-07-03-53-37_rotation_linear_mob_patch.mat'

file_name6 = '/mnt/data/WQ/Argos/train_18_test_6_antenna_96_subcarriers_52/train/downlink/ArgosCSI-96x2-2016-03-31-15-53-27_Jian_left_to_right.mat'

ant_idx = 2
data = np.load(file_name)[:5026, 0, ant_idx, :]
data2 = np.load(file_name2)[:5026, 0, ant_idx, :]
data3 = np.load(file_name3)[:5026, 0, ant_idx, :]
data4 = loadmat(file_name4)['data'][:5026, 0, ant_idx, :]
data5 = loadmat(file_name5)['data'][:5026, 0, ant_idx, :]
data6 = loadmat(file_name6)['data'][:5026, 0, ant_idx, :]

print(data.shape, data.dtype)
print(data2.shape, data2.dtype)
print(data3.shape, data3.dtype)
print(data4.shape, data4.dtype)
print(data5.shape, data5.dtype)
print(data6.shape, data6.dtype)


(5026, 52) complex128
(5026, 52) complex128
(5026, 52) complex128
(5026, 52) complex64
(5026, 52) complex64
(5026, 52) complex64


In [2]:
known_env = [data, data2, data3, data5, data6]
unknown_env = [data4]


In [3]:
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from scipy.stats import wasserstein_distance

# 计算KL散度的函数
def calculate_kl_divergence(p, q, epsilon=1e-10):
    """
    计算两个概率分布之间的KL散度
    """
    # 添加小的epsilon值以避免除零错误
    p = p + epsilon
    q = q + epsilon
    
    # 归一化为概率分布
    p = p / np.sum(p)
    q = q / np.sum(q)
    
    # 计算KL散度
    return entropy(p, q)

# 将复数数据转换为幅度
def complex_to_magnitude(data):
    """
    将复数数据转换为幅度
    """
    return np.abs(data)

# 计算所有known_env数据对之间的KL散度
kl_divergences = []
js_divergences = []
wasserstein_distances = []

print("计算known_env中两两之间数据的KL散度...")

# 将复数数据转换为幅度并展平
magnitude_data = []
for i, env_data in enumerate(known_env):
    mag_data = complex_to_magnitude(env_data)
    # 展平数据以便计算KL散度
    flattened_data = mag_data.flatten()
    magnitude_data.append(flattened_data)
    print(f"data{i+1} shape: {flattened_data.shape}")

# 计算两两之间的KL散度
for i in range(len(magnitude_data)):
    for j in range(i+1, len(magnitude_data)):
        # 确保两个分布具有相同的长度
        min_len = min(len(magnitude_data[i]), len(magnitude_data[j]))
        p = magnitude_data[i][:min_len]
        q = magnitude_data[j][:min_len]
        
        # 计算KL散度 (D_KL(P||Q) 和 D_KL(Q||P))
        kl_pq = calculate_kl_divergence(p, q)
        kl_qp = calculate_kl_divergence(q, p)
        
        # 计算对称KL散度
        sym_kl = (kl_pq + kl_qp) / 2
        
        # 计算JS散度作为另一种度量
        js_div = jensenshannon(p/np.sum(p), q/np.sum(q))**2
        js_divergences.append({
            'pair': (i+1, j+1),
            'JS divergence': js_div
        })

        w_dist = wasserstein_distance(p, q)
        wasserstein_distances.append({
            'pair': (i+1, j+1),
            'Wasserstein-1 distance': w_dist
        })
        
        print(f"Wasserstein-1距离 between data{i+1} and data{j+1}: {w_dist:.6f}")

        kl_divergences.append({
            'pair': (i+1, j+1),
            'KL(P||Q)': kl_pq,
            'KL(Q||P)': kl_qp,
            'Symmetric KL': sym_kl,
            'JS divergence': js_div
        })
        
        print(f"KL散度 between data{i+1} and data{j+1}:")
        print(f"  D_KL(P||Q) = {kl_pq:.6f}")
        print(f"  D_KL(Q||P) = {kl_qp:.6f}")
        print(f"  Symmetric KL = {sym_kl:.6f}")
        print(f"  JS divergence = {js_div:.6f}")
        print(f"Wasserstein-1距离 between data{i+1} and data{j+1}: {w_dist:.6f}")
        print()

print("KL散度计算完成！")

计算known_env中两两之间数据的KL散度...
data1 shape: (261352,)
data2 shape: (261352,)
data3 shape: (261352,)
data4 shape: (261352,)
data5 shape: (261352,)
Wasserstein-1距离 between data1 and data2: 1.090226
KL散度 between data1 and data2:
  D_KL(P||Q) = 0.465811
  D_KL(Q||P) = 0.528009
  Symmetric KL = 0.496910
  JS divergence = 0.108061
Wasserstein-1距离 between data1 and data2: 1.090226

Wasserstein-1距离 between data1 and data3: 0.726983
KL散度 between data1 and data3:
  D_KL(P||Q) = 0.242270
  D_KL(Q||P) = 0.250615
  Symmetric KL = 0.246442
  JS divergence = 0.056328
Wasserstein-1距离 between data1 and data3: 0.726983

Wasserstein-1距离 between data1 and data4: 0.950015
KL散度 between data1 and data4:
  D_KL(P||Q) = 0.527138
  D_KL(Q||P) = 0.510520
  Symmetric KL = 0.518829
  JS divergence = 0.115493
Wasserstein-1距离 between data1 and data4: 0.950015

Wasserstein-1距离 between data1 and data5: 0.739929
KL散度 between data1 and data5:
  D_KL(P||Q) = 0.147483
  D_KL(Q||P) = 0.172578
  Symmetric KL = 0.160031
  JS div

In [4]:
import torch
import torch.nn.functional as F
model_path = [f'/mnt/data/WQ/LoRAT/model/exp_Argos_4_30/FT_Client{i}/lora/lora_weights.pt' for i in [0, 1, 2, 3, 5]]
known_models = [torch.load(path) for path in model_path]

In [5]:
# 计算模型之间相似度的函数
def flatten_model_weights(model_state_dict):
    """
    将模型权重展平为一维向量
    """
    flattened_weights = []
    for key in sorted(model_state_dict.keys()):  # 排序以确保一致性
        flattened_weights.append(model_state_dict[key].flatten())
    return torch.cat(flattened_weights)

# 计算normalized l1/l2 distance和cosine similarity
model_similarities = []

print("计算known_models中两两模型之间的相似度...")

# 将模型权重展平
flattened_models = []
for i, model in enumerate(known_models):
    flattened_model = flatten_model_weights(model)
    flattened_models.append(flattened_model)
    print(f"model{i} flattened shape: {flattened_model.shape}")

# 计算两两之间的相似度
for i in range(len(flattened_models)):
    for j in range(i+1, len(flattened_models)):
        model1 = flattened_models[i]
        model2 = flattened_models[j]
        
        # 计算l1距离
        l1_distance = torch.norm(model1 - model2, p=1).item()
        
        # 计算l2距离
        l2_distance = torch.norm(model1 - model2, p=2).item()
        
        # 计算normalized l1距离
        normalized_l1 = l1_distance / len(model1)
        
        # 计算normalized l2距离
        normalized_l2 = l2_distance / np.sqrt(len(model1))
        
        # 计算cosine similarity
        cosine_sim = F.cosine_similarity(model1.unsqueeze(0), model2.unsqueeze(0)).item()
        
        model_similarities.append({
            'pair': (i, j),
            'l1_distance': l1_distance,
            'l2_distance': l2_distance,
            'normalized_l1': normalized_l1,
            'normalized_l2': normalized_l2,
            'cosine_similarity': cosine_sim
        })
        
        print(f"模型 {i} 和模型 {j} 之间的相似度:")
        print(f"  L1距离: {l1_distance:.6f}")
        print(f"  L2距离: {l2_distance:.6f}")
        print(f"  Normalized L1距离: {normalized_l1:.6f}")
        print(f"  Normalized L2距离: {normalized_l2:.6f}")
        print(f"  Cosine相似度: {cosine_sim:.6f}")
        print()

# # 保存结果
# import json
# def convert_tensor(obj):
#     if isinstance(obj, torch.Tensor):
#         return obj.item()
#     raise TypeError(f'Object of type {type(obj)} is not JSON serializable')

# with open('/mnt/data/WQ/LoRAT/model_similarities.json', 'w') as f:
#     json.dump(model_similarities, f, indent=2, default=convert_tensor)

print("模型相似度计算完成，结果已保存到 /mnt/data/WQ/LoRAT/model_similarities.json")

计算known_models中两两模型之间的相似度...
model0 flattened shape: torch.Size([294912])
model1 flattened shape: torch.Size([294912])
model2 flattened shape: torch.Size([294912])
model3 flattened shape: torch.Size([294912])
model4 flattened shape: torch.Size([294912])
模型 0 和模型 1 之间的相似度:
  L1距离: 9266.748047
  L2距离: 36.498940
  Normalized L1距离: 0.031422
  Normalized L2距离: 0.067210
  Cosine相似度: 0.472261

模型 0 和模型 2 之间的相似度:
  L1距离: 7685.079102
  L2距离: 31.785244
  Normalized L1距离: 0.026059
  Normalized L2距离: 0.058530
  Cosine相似度: 0.692073

模型 0 和模型 3 之间的相似度:
  L1距离: 8851.396484
  L2距离: 36.846680
  Normalized L1距离: 0.030014
  Normalized L2距离: 0.067850
  Cosine相似度: 0.528995

模型 0 和模型 4 之间的相似度:
  L1距离: 8375.241211
  L2距离: 32.722618
  Normalized L1距离: 0.028399
  Normalized L2距离: 0.060256
  Cosine相似度: 0.543656

模型 1 和模型 2 之间的相似度:
  L1距离: 9899.349609
  L2距离: 38.820587
  Normalized L1距离: 0.033567
  Normalized L2距离: 0.071485
  Cosine相似度: 0.459963

模型 1 和模型 3 之间的相似度:
  L1距离: 8740.822266
  L2距离: 34.703926
  Normali

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os

# 创建保存目录
os.makedirs('/mnt/data/WQ/LoRAT/mapping_model', exist_ok=True)

# 定义神经网络模型
class DistributionToModelMapper(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DistributionToModelMapper, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim//2)
        self.fc4 = nn.Linear(hidden_dim//2, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# 准备数据
print("准备训练数据...")

# 构建输入特征和输出目标
input_features = []
output_targets = []
data_pairs = []
model_pairs = []

# 收集数据分布散度和模型距离
for i, kl_div in enumerate(kl_divergences):
    pair = kl_div['pair']
    kl_pq = kl_div['KL(P||Q)']
    kl_qp = kl_div['KL(Q||P)']
    sym_kl = kl_div['Symmetric KL']
    js_div = js_divergences[i]['JS divergence']
    w_dist = wasserstein_distances[i]['Wasserstein-1 distance']
    
    # 收集模型相似度
    model_sim = model_similarities[i]
    model_pair = model_sim['pair']
    l1_dist = model_sim['l1_distance']
    l2_dist = model_sim['l2_distance']
    norm_l1 = model_sim['normalized_l1']
    norm_l2 = model_sim['normalized_l2']
    cosine_sim = model_sim['cosine_similarity']
    
    # 确保数据对齐
    if pair == (model_pair[0]+1, model_pair[1]+1):  # 数据索引从1开始，模型索引从0开始
        # 输入特征：[KL(P||Q), KL(Q||P), Symmetric KL, JS divergence, Wasserstein-1 distance]
        input_features.append([kl_pq, kl_qp, sym_kl, js_div, w_dist])
        # 输出目标：[L1距离, L2距离, Normalized L1, Normalized L2, Cosine相似度]
        output_targets.append([l1_dist, l2_dist, norm_l1, norm_l2, cosine_sim])
        
        data_pairs.append(pair)
        model_pairs.append(model_pair)

# 转换为numpy数组
X = np.array(input_features)
y = np.array(output_targets)

print(f"输入特征形状: {X.shape}")
print(f"输出目标形状: {y.shape}")

# 定义不同的输入输出组合
input_names = ['KL(P||Q)', 'KL(Q||P)', 'Symmetric KL', 'JS divergence', 'Wasserstein-1 distance']
output_names = ['L1 distance', 'L2 distance', 'Normalized L1', 'Normalized L2', 'Cosine similarity']

# 训练多个模型，每个模型使用一种输入特征预测一种输出目标
trained_models = {}
scalers = {}

print("\n开始训练多个神经网络模型...")

# 为每种输入输出组合训练一个模型
for input_idx, input_name in enumerate(input_names):
    for output_idx, output_name in enumerate(output_names):
        print(f"\n训练模型: {input_name} -> {output_name}")
        
        # 准备数据
        X_single = X[:, input_idx].reshape(-1, 1)  # 只使用一种输入特征
        y_single = y[:, output_idx].reshape(-1, 1)  # 只预测一种输出目标
        
        # 数据标准化
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X_scaled = scaler_X.fit_transform(X_single)
        y_scaled = scaler_y.fit_transform(y_single)
        
        # 划分训练集和测试集
        # X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.0, random_state=42)
        X_train, y_train = X_scaled, y_scaled
        
        # 转换为PyTorch张量
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        # X_test_tensor = torch.FloatTensor(X_test)
        # y_test_tensor = torch.FloatTensor(y_test)
        
        # 初始化模型
        model = DistributionToModelMapper(1, 32, 1)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        
        # 训练模型
        num_epochs = 500
        train_losses = []
        test_losses = []
        
        for epoch in range(num_epochs):
            # 训练阶段
            model.train()
            optimizer.zero_grad()
            outputs = model(X_train_tensor)
            loss = criterion(outputs, y_train_tensor)
            loss.backward()
            optimizer.step()
            
            # # 测试阶段
            # model.eval()
            # with torch.no_grad():
            #     test_outputs = model(X_test_tensor)
            #     test_loss = criterion(test_outputs, y_test_tensor)
            
            train_losses.append(loss.item())
            # test_losses.append(test_loss.item())
            
            # 每100个epoch打印一次
            if (epoch + 1) % 100 == 0:
                print(f'  Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.6f}')
        
        # 评估模型
        model.eval()
        with torch.no_grad():
            train_predictions = model(X_train_tensor)
            # test_predictions = model(X_test_tensor)
            
            train_mse = criterion(train_predictions, y_train_tensor)
            # test_mse = criterion(test_predictions, y_test_tensor)
            
            print(f"  训练集MSE: {train_mse.item():.6f}")
            # print(f"  测试集MSE: {test_mse.item():.6f}")
        
        # 保存模型和缩放器
        model_key = f"{input_name.replace(' ', '_')}_{output_name.replace(' ', '_')}"
        trained_models[model_key] = model
        scalers[model_key] = (scaler_X, scaler_y)
        
        # 保存模型
        torch.save({
            'model_state_dict': model.state_dict(),
            'scaler_X': scaler_X,
            'scaler_y': scaler_y,
        }, f'/mnt/data/WQ/LoRAT/mapping_model/{model_key}.pth')
        
        print(f"  模型已保存: {model_key}.pth")

print("\n所有模型训练完成!")


准备训练数据...
输入特征形状: (10, 5)
输出目标形状: (10, 5)

开始训练多个神经网络模型...

训练模型: KL(P||Q) -> L1 distance
  Epoch [100/500], Train Loss: 0.499674
  Epoch [200/500], Train Loss: 0.206161
  Epoch [300/500], Train Loss: 0.129086
  Epoch [400/500], Train Loss: 0.105408
  Epoch [500/500], Train Loss: 0.048097
  训练集MSE: 0.082996
  模型已保存: KL(P||Q)_L1_distance.pth

训练模型: KL(P||Q) -> L2 distance
  Epoch [100/500], Train Loss: 0.109878
  Epoch [200/500], Train Loss: 0.087063
  Epoch [300/500], Train Loss: 0.100814
  Epoch [400/500], Train Loss: 0.074222
  Epoch [500/500], Train Loss: 0.115106
  训练集MSE: 0.063785
  模型已保存: KL(P||Q)_L2_distance.pth

训练模型: KL(P||Q) -> Normalized L1
  Epoch [100/500], Train Loss: 0.240038
  Epoch [200/500], Train Loss: 0.109449
  Epoch [300/500], Train Loss: 0.216100
  Epoch [400/500], Train Loss: 0.105610
  Epoch [500/500], Train Loss: 0.093667
  训练集MSE: 0.076391
  模型已保存: KL(P||Q)_Normalized_L1.pth

训练模型: KL(P||Q) -> Normalized L2
  Epoch [100/500], Train Loss: 0.104171
  Epoch [200

In [8]:

# 计算unknown_env与known_env之间的分布散度
print("\n计算unknown_env与known_env之间的分布散度...")

# 将unknown_env数据转换为幅度并展平
unknown_magnitude_data = []
for i, env_data in enumerate(unknown_env):
    mag_data = complex_to_magnitude(env_data)
    flattened_data = mag_data.flatten()
    unknown_magnitude_data.append(flattened_data)
    print(f"unknown_env{i} shape: {flattened_data.shape}")

# 计算unknown_env与known_env之间的分布散度
unknown_divergences = []
for i, unknown_data in enumerate(unknown_magnitude_data):
    for j, known_data in enumerate(magnitude_data):
        # 确保两个分布具有相同的长度
        min_len = min(len(unknown_data), len(known_data))
        p = unknown_data[:min_len]
        q = known_data[:min_len]
        
        # 计算各种散度
        kl_pq = calculate_kl_divergence(p, q)
        kl_qp = calculate_kl_divergence(q, p)
        sym_kl = (kl_pq + kl_qp) / 2
        js_div = jensenshannon(p/np.sum(p), q/np.sum(q))**2
        w_dist = wasserstein_distance(p, q)
        
        unknown_divergences.append({
            'pair': (f'unknown_env{i}', f'known_env{j}'),
            'KL(P||Q)': kl_pq,
            'KL(Q||P)': kl_qp,
            'Symmetric KL': sym_kl,
            'JS divergence': js_div,
            'Wasserstein-1 distance': w_dist
        })
        
        print(f"unknown_env{i} 与 known_env{j} 之间的散度:")
        print(f"  KL(P||Q) = {kl_pq:.6f}")
        print(f"  KL(Q||P) = {kl_qp:.6f}")
        print(f"  Symmetric KL = {sym_kl:.6f}")
        print(f"  JS divergence = {js_div:.6f}")
        print(f"  Wasserstein-1 distance = {w_dist:.6f}")
        print()

# 使用训练好的模型预测unknown_env与known_env之间的模型距离
print("\n使用训练好的模型预测unknown_env与known_env之间的模型距离...")

predictions = {}
for i, divergence in enumerate(unknown_divergences):
    pair = divergence['pair']
    print(f"\n预测 {pair[0]} 与 {pair[1]} 之间的模型距离:")
    
    predictions[pair] = {}
    
    # 使用每种输入特征进行预测
    for input_idx, input_name in enumerate(input_names):
        input_value = divergence[input_name]
        predictions[pair][input_name] = {}
        
        # 使用每种输出目标的模型进行预测
        for output_idx, output_name in enumerate(output_names):
            model_key = f"{input_name.replace(' ', '_')}_{output_name.replace(' ', '_')}"
            
            # 加载模型和缩放器
            if model_key in trained_models:
                model = trained_models[model_key]
                scaler_X, scaler_y = scalers[model_key]
                
                # 标准化输入
                input_scaled = scaler_X.transform([[input_value]])
                input_tensor = torch.FloatTensor(input_scaled)
                
                # 预测
                model.eval()
                with torch.no_grad():
                    prediction_scaled = model(input_tensor)
                    # 反标准化
                    prediction = scaler_y.inverse_transform(prediction_scaled.numpy())[0, 0]
                
                predictions[pair][input_name][output_name] = prediction
                print(f"  {input_name} -> {output_name}: {prediction:.6f}")

save_prediction = {}
for pair, input_predictions in predictions.items():
    save_prediction[str(pair)] = input_predictions

# 保存预测结果
import json
with open('/mnt/data/WQ/LoRAT/unknown_env_predictions.json', 'w') as f:
    json.dump(save_prediction, f, indent=2, default=float)

print("\n预测结果已保存到 /mnt/data/WQ/LoRAT/unknown_env_predictions.json")

# 总结预测结果
print("\n预测结果总结:")
for pair, input_predictions in predictions.items():
    print(f"\n{pair[0]} 与 {pair[1]} 之间的预测模型距离:")
    for input_name, output_predictions in input_predictions.items():
        print(f"  基于 {input_name} 的预测:")
        for output_name, prediction in output_predictions.items():
            print(f"    {output_name}: {prediction:.6f}")


计算unknown_env与known_env之间的分布散度...
unknown_env0 shape: (261352,)
unknown_env0 与 known_env0 之间的散度:
  KL(P||Q) = 0.402975
  KL(Q||P) = 0.377520
  Symmetric KL = 0.390248
  JS divergence = 0.087166
  Wasserstein-1 distance = 0.525469

unknown_env0 与 known_env1 之间的散度:
  KL(P||Q) = 0.435871
  KL(Q||P) = 0.438831
  Symmetric KL = 0.437351
  JS divergence = 0.097759
  Wasserstein-1 distance = 0.567214

unknown_env0 与 known_env2 之间的散度:
  KL(P||Q) = 0.329105
  KL(Q||P) = 0.313058
  Symmetric KL = 0.321081
  JS divergence = 0.072202
  Wasserstein-1 distance = 0.205136

unknown_env0 与 known_env3 之间的散度:
  KL(P||Q) = 0.433360
  KL(Q||P) = 0.417531
  Symmetric KL = 0.425445
  JS divergence = 0.095445
  Wasserstein-1 distance = 0.424546

unknown_env0 与 known_env4 之间的散度:
  KL(P||Q) = 0.212722
  KL(Q||P) = 0.242385
  Symmetric KL = 0.227554
  JS divergence = 0.053372
  Wasserstein-1 distance = 0.501348


使用训练好的模型预测unknown_env与known_env之间的模型距离...

预测 unknown_env0 与 known_env0 之间的模型距离:
  KL(P||Q) -> L1 d

In [9]:
import numpy as np
import json

# 读取预测结果
with open('/mnt/data/WQ/LoRAT/unknown_env_predictions.json', 'r') as f:
    predictions = json.load(f)

print("将模型距离转换为概率分布...")

# 定义softmax函数
def softmax(x):
    """计算softmax值"""
    e_x = np.exp(x - np.max(x))  # 减去最大值以避免数值溢出
    return e_x / e_x.sum()

# 定义高斯核函数
def gaussian_kernel(x, sigma=1.0):
    """使用高斯核将距离转换为相似度"""
    return np.exp(-x**2 / (2 * sigma**2))

# 将距离转换为概率分布的函数
def distance_to_probability(distances, method='softmax'):
    """
    将距离转换为概率分布
    method: 'softmax', 'gaussian', 'inverse'
    """
    if method == 'softmax':
        # 使用softmax，距离越小概率越大
        # 对距离取负值，因为softmax通常用于最大化
        return softmax(-np.array(distances))
    elif method == 'gaussian':
        # 使用高斯核函数
        similarities = gaussian_kernel(np.array(distances))
        return similarities / np.sum(similarities)
    elif method == 'inverse':
        # 使用反比函数
        # 添加小的epsilon值以避免除零错误
        epsilon = 1e-8
        inverse_distances = 1.0 / (np.array(distances) + epsilon)
        return inverse_distances / np.sum(inverse_distances)
    else:
        raise ValueError("Unknown method. Use 'softmax', 'gaussian', or 'inverse'.")

# 对于每个unknown_env，收集与所有known_env的模型距离
unknown_env_probabilities = {}

# 定义要使用的输入特征和输出目标
input_feature = 'Symmetric KL'  # 可以更改为您想要使用的特征
output_target = 'Cosine similarity'  # 可以更改为您想要使用的输出目标

print(f"\n使用 {input_feature} 预测 {output_target} 来计算概率分布")

# 按unknown_env分组收集距离
grouped_distances = {}
for pair_str, input_predictions in predictions.items():
    # 解析pair字符串
    pair = eval(pair_str)
    unknown_env_name = pair[0]
    known_env_name = pair[1]
    
    # 获取预测的距离
    if input_feature in input_predictions and output_target in input_predictions[input_feature]:
        distance = input_predictions[input_feature][output_target]
        
        # 按unknown_env分组
        if unknown_env_name not in grouped_distances:
            grouped_distances[unknown_env_name] = []
        grouped_distances[unknown_env_name].append((known_env_name, distance))

# 对每个unknown_env计算概率分布
for unknown_env_name, distances in grouped_distances.items():
    print(f"\n{unknown_env_name} 与各known_env的模型距离:")
    
    # 提取距离值
    distance_values = [dist[1] for dist in distances]
    known_env_names = [dist[0] for dist in distances]
    
    # 显示距离
    for i, (known_env, dist) in enumerate(distances):
        print(f"  {known_env}: {dist:.6f}")
    
    # 使用不同方法计算概率分布
    softmax_probs = distance_to_probability(distance_values, method='softmax')
    gaussian_probs = distance_to_probability(distance_values, method='gaussian')
    inverse_probs = distance_to_probability(distance_values, method='inverse')
    
    # 保存结果
    unknown_env_probabilities[unknown_env_name] = {
        'known_envs': known_env_names,
        'distances': distance_values,
        'softmax_probabilities': softmax_probs.tolist(),
        'gaussian_probabilities': gaussian_probs.tolist(),
        'inverse_probabilities': inverse_probs.tolist()
    }
    
    # 显示概率分布
    print(f"\n{unknown_env_name} 与各known_env的相似度概率分布:")
    print("  Softmax方法:")
    for i, (known_env, prob) in enumerate(zip(known_env_names, softmax_probs)):
        print(f"    {known_env}: {prob:.6f}")
    
    print("  高斯核方法:")
    for i, (known_env, prob) in enumerate(zip(known_env_names, gaussian_probs)):
        print(f"    {known_env}: {prob:.6f}")
        
    print("  反比方法:")
    for i, (known_env, prob) in enumerate(zip(known_env_names, inverse_probs)):
        print(f"    {known_env}: {prob:.6f}")

# 保存概率分布结果
with open('/mnt/data/WQ/LoRAT/unknown_env_probabilities.json', 'w') as f:
    json.dump(unknown_env_probabilities, f, indent=2)

print("\n概率分布结果已保存到 /mnt/data/WQ/LoRAT/unknown_env_probabilities.json")

# 找到每个unknown_env最相似的known_env
print("\n最相似的known_env:")
for unknown_env_name, probs in unknown_env_probabilities.items():
    known_envs = probs['known_envs']
    softmax_probs = np.array(probs['softmax_probabilities'])
    gaussian_probs = np.array(probs['gaussian_probabilities'])
    inverse_probs = np.array(probs['inverse_probabilities'])
    
    # 找到最大概率的索引
    softmax_max_idx = np.argmax(softmax_probs)
    gaussian_max_idx = np.argmax(gaussian_probs)
    inverse_max_idx = np.argmax(inverse_probs)
    
    print(f"\n{unknown_env_name} 最相似的known_env:")
    print(f"  基于Softmax: {known_envs[softmax_max_idx]} (概率: {softmax_probs[softmax_max_idx]:.6f})")
    print(f"  基于高斯核: {known_envs[gaussian_max_idx]} (概率: {gaussian_probs[gaussian_max_idx]:.6f})")
    print(f"  基于反比: {known_envs[inverse_max_idx]} (概率: {inverse_probs[inverse_max_idx]:.6f})")

将模型距离转换为概率分布...

使用 Symmetric KL 预测 Cosine similarity 来计算概率分布

unknown_env0 与各known_env的模型距离:
  known_env0: 0.457882
  known_env1: 0.466342
  known_env2: 0.647126
  known_env3: 0.463236
  known_env4: 0.597637

unknown_env0 与各known_env的相似度概率分布:
  Softmax方法:
    known_env0: 0.213521
    known_env1: 0.211722
    known_env2: 0.176706
    known_env3: 0.212380
    known_env4: 0.185671
  高斯核方法:
    known_env0: 0.207329
    known_env1: 0.206520
    known_env2: 0.186746
    known_env3: 0.206818
    known_env4: 0.192587
  反比方法:
    known_env0: 0.225022
    known_env1: 0.220940
    known_env2: 0.159217
    known_env3: 0.222421
    known_env4: 0.172401

概率分布结果已保存到 /mnt/data/WQ/LoRAT/unknown_env_probabilities.json

最相似的known_env:

unknown_env0 最相似的known_env:
  基于Softmax: known_env0 (概率: 0.213521)
  基于高斯核: known_env0 (概率: 0.207329)
  基于反比: known_env0 (概率: 0.225022)


In [56]:
import torch

print("\n使用softmax概率分布作为系数聚合模型...")

# 读取概率分布结果
with open('/mnt/data/WQ/LoRAT/unknown_env_probabilities.json', 'r') as f:
    unknown_env_probabilities = json.load(f)

# 为每个unknown_env聚合模型
aggregated_models = {}

for unknown_env_name, probs in unknown_env_probabilities.items():
    print(f"\n为 {unknown_env_name} 聚合模型...")
    
    # 获取softmax概率
    softmax_probs = np.array(probs['softmax_probabilities'])
    known_env_names = probs['known_envs']
    
    print(f"  使用的聚合系数 (softmax概率):")
    for i, (known_env, prob) in enumerate(zip(known_env_names, softmax_probs)):
        print(f"    {known_env}: {prob:.6f}")
    
    # 将known_env名称转换为索引
    # known_env0, known_env1, known_env2, known_env3, known_env4 对应索引 0, 1, 2, 3, 4
    known_env_indices = []
    for known_env in known_env_names:
        # 从名称中提取索引
        idx = int(known_env.replace('known_env', ''))
        known_env_indices.append(idx)
    
    # 获取对应的模型
    models_to_aggregate = [known_models[i] for i in known_env_indices]
    
    # 创建聚合模型
    aggregated_model = {}
    
    # 获取第一个模型的键作为参考
    reference_model = models_to_aggregate[0]
    
    # 对每个参数进行加权平均
    for key in reference_model.keys():
        # 初始化聚合参数
        aggregated_param = torch.zeros_like(reference_model[key])
        
        # 对所有模型的参数进行加权平均
        for i, model in enumerate(models_to_aggregate):
            weight = softmax_probs[i]
            aggregated_param += weight * model[key]
        
        # 保存聚合后的参数
        aggregated_model[key] = aggregated_param
    
    # 保存聚合模型
    aggregated_models[unknown_env_name] = aggregated_model
    
    distance_name = input_feature.split(' ')[0] + '_' + output_target.split(' ')[0]
    # 保存到文件
    save_path = f'/mnt/data/WQ/LoRAT/mapping_model/aggregated_model_{distance_name}.pt'
    torch.save(aggregated_model, save_path)
    print(f"  聚合模型已保存到: {save_path}")

print("\n所有模型聚合完成!")

# 验证聚合模型
print("\n验证聚合模型...")
for unknown_env_name, aggregated_model in aggregated_models.items():
    print(f"\n{unknown_env_name} 的聚合模型参数:")
    for key, param in list(aggregated_model.items())[:3]:  # 只显示前3个参数
        print(f"  {key}: shape {param.shape}, mean {param.mean().item():.6f}, std {param.std().item():.6f}")
    if len(aggregated_model) > 3:
        print(f"  ... (共{len(aggregated_model)}个参数)")


使用softmax概率分布作为系数聚合模型...

为 unknown_env0 聚合模型...
  使用的聚合系数 (softmax概率):
    known_env0: 0.213834
    known_env1: 0.209452
    known_env2: 0.171295
    known_env3: 0.212712
    known_env4: 0.192707
  聚合模型已保存到: /mnt/data/WQ/LoRAT/mapping_model/aggregated_model_Symmetric_Cosine.pt

所有模型聚合完成!

验证聚合模型...

unknown_env0 的聚合模型参数:
  spatial_block.blocks.0.s_attn.w_q.lora_A_r: shape torch.Size([8, 128]), mean 0.001781, std 0.050632
  spatial_block.blocks.0.s_attn.w_q.lora_B_r: shape torch.Size([128, 8]), mean 0.000000, std 0.000000
  spatial_block.blocks.0.s_attn.w_q.lora_A_i: shape torch.Size([8, 128]), mean 0.001954, std 0.050310
  ... (共288个参数)
